---
#### 작업형1-1. 주어진 데이터에서 views 컬럼을 기준으로 내림차순 정렬하고, views 컬럼 상위 10개 데이터를 views 컬럼 상위 10번째 값으로 대체한 후, age컬럼이 80 이상인 데이터의 views컬럼 평균값을 구하라 (반올림 하여 소수 2째자리)
---

In [10]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch2/members.csv")
print(df.shape)
df.head()

(100, 10)


,id,age,city,f1,f2,f3,f4,f5,subscribed,views
0,id01,2.0,서울,NaN,0,gold,ENFJ,91.297791,2021-07-16,6820.0
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826,2021-05-12,2534.0
2,id03,27.0,서울,61.0,1,gold,ISTJ,17.252986,2021-03-16,7312.0
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078,2021-07-21,493.0
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869,2021-03-07,1338.0


In [12]:
df_sorted = df.sort_values('views', ascending=False).reset_index()
view_10th = 9690.0
df_sorted.iloc[:10, -1] = view_10th
temp = df_sorted[df_sorted['age']>= 80]
print('정답 = ', round(temp['views'].mean(), 2))

정답 =  5674.04


---
#### 작업형1-2. 주어진 데이터셋에서 앞에서 부터 순서대로 80% 데이터만 활용해 f1컬럼 결측치를 중앙값으로 채우기 전후의 표본 표준편차를 구하고, 두 표본 표준편차의 차이를 계산하라 (절대값으로 계산, 소수 2째자라 반올림)
---

In [25]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch2/members.csv")

In [26]:
df_80 = df[:int(len(df)*0.8)]
before_std = df_80['f1'].std()

df_80['f1'] = df_80['f1'].fillna(df_80['f1'].median())
after_std = df_80['f1'].std()

print('정답 = ', round(abs(before_std - after_std), 2))

정답 =  3.56


/tmp/ipython-input-4157008723.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_80['f1'] = df_80['f1'].fillna(df_80['f1'].median())


---
#### 작업형 1-3. 주어진 데이터셋의 age 컬럼의 이상치를 모두 더하라. 이상치는 평균으로 부터 표준편차 * 1.5를 벗어나는 영역임
---

In [27]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch2/members.csv")

In [29]:
age_mean = df['age'].mean()
age_std = df['age'].std()
lower = age_mean - (1.5*age_std)
upper = age_mean + (1.5*age_std)

df_outlier = df[
    (df['age'] < lower) | (df['age'] > upper)
]

print('정답 = ', df_outlier['age'].sum())

정답 =  473.5


---
#### 작업형 2-1. 제품 배송시간에 맞춰 배송되지 않을 확률값을 예측하라
- 타겟 : Reached.on.Time_Y.N(0:정시도착, 1:정시미도착)
- 평가 : roc-auc
- 제출 : result.csv (컬럼: ID, Reached.on.Time_Y.N)
---

In [77]:
import pandas as pd
X_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch2/X_test.csv")
X_train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch2/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch2/y_train.csv")
print(X_train.shape, X_test.shape, y_train.shape)
display(X_train.head(), y_train.head())

######## EDA #######################
# 수치형, 범주형 변수 혼합
# 결측치 없음, 범주형 변수들의 train/test간 카테고리는 동일함
####################################

# 1. ID삭제, 원-핫 인코딩
X_train = X_train.drop('ID', axis=1)
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# 2. 검증데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train['Reached.on.Time_Y.N'], test_size=0.2, random_state=42)

# 3. model, eval
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

model = RandomForestClassifier(random_state=120)
model.fit(X_train, y_train)

model_pred = model.predict_proba(X_valid)[:,1]
score = roc_auc_score(y_valid, model_pred)
print('ROC AUC = ', score)

# 4. submit
X_test_id = X_test['ID']
X_test = X_test.drop('ID', axis=1)

pred = model.predict_proba(X_test)
result = pd.DataFrame({
    'ID':X_test_id, 'Reached.on.Time_Y.N':pred[:, 1]
})
result.to_csv('result.csv', index=False)

(8799, 11) (2200, 11) (8799, 2)


,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,8440,B,Ship,4,1,284,4,low,F,4,4795
1,10849,D,Ship,3,2,201,4,medium,M,10,5565
2,9478,B,Ship,4,4,181,3,medium,F,9,4425
3,2968,B,Ship,6,2,236,4,low,F,61,3140
4,6964,B,Road,5,1,282,4,low,F,9,4766


,ID,Reached.on.Time_Y.N
0,8440,1
1,10849,0
2,9478,1
3,2968,1
4,6964,0


ROC AUC =  0.7476043889223687
